##### Copyright 2020 The TensorFlow IO Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 颜色空间转换

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/io/tutorials/colorspace"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/io/blob/master/docs/tutorials/colorspace.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/io/blob/master/docs/tutorials/colorspace.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
      <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/io/docs/tutorials/colorspace.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## 概述

在计算机视觉中，所选颜色空间可能对模型的性能产生重大影响。虽然 `RGB` 是最常用的颜色空间，但是在许多情况下，切换至其他颜色空间（例如 `YUV`、`YCbCr`、`XYZ (CIE)` 等）时，模型的性能可能更出色。

`tensorflow-io` 软件包提供了一系列可用于准备和增强图像数据的颜色空间转换 API。

## 设置

### 安装要求的软件包，然后重新启动运行时

In [ ]:
!pip install tensorflow-io

### 下载示例图像

本教程中使用的示例图像是一张[站在雪地中的猫](https://commons.wikimedia.org/wiki/File:Felis_catus-cat_on_snow.jpg)的照片，不过，您可以将其替换为任何 JPEG 图像。

下面的代码会下载这个图像，并将其以 `sample.jpg` 形式保存到本地磁盘上：

In [ ]:
!curl -o sample.jpg -L https://storage.googleapis.com/download.tensorflow.org/example_images/320px-Felis_catus-cat_on_snow.jpg

!ls -ls sample.jpg

## 用法

### 读取图像文件

读取示例图像并将其解码成形状为 `(213, 320, 3)` 的 `uint8` 张量

In [ ]:
import tensorflow as tf
import tensorflow_io as tfio

image = tf.image.decode_jpeg(tf.io.read_file('sample.jpg'))

print(image.shape, image.dtype)

通过以下代码可以显示该图像：

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(image)
plt.axis('off')
plt.show()

### 将 RGB 转换为灰度图像

您可以利用 `tfio.experimental.color.rgb_to_grayscale` 将 `RGB` 图像转换为 `Grayscale` 图像，以将颜色通道从 3 个减少为 1 个：

In [ ]:
grayscale = tfio.experimental.color.rgb_to_grayscale(image)

print(grayscale.shape, grayscale.dtype)

# use tf.squeeze to remove last channel for plt.imshow to display:
plt.figure()
plt.imshow(tf.squeeze(grayscale, axis=-1), cmap='gray')
plt.axis('off')
plt.show()

### 将 RGB 转换为 BGR

某些图像软件和相机制造商可能更喜欢使用 `BGR`，通过 `tfio.experimental.color.rgb_to_bgr` 即可获取这种格式：

In [ ]:
bgr = tfio.experimental.color.rgb_to_bgr(image)

print(bgr.shape, bgr.dtype)

plt.figure()
plt.imshow(bgr)
plt.axis('off')
plt.show()

### 将 RGB 转换为 CIE XYZ

`CIE XYZ`（或称 `CIE 1931 XYZ`）是在很多图像处理程序中常用的一种颜色空间。下面是通过 `tfio.experimental.color.rgb_to_xyz` 从 RGB 转换为 `CIE XYZ` 的代码。请注意，`tfio.experimental.color.rgb_to_xyz` 假设输入为 `[0, 1]` 范围内的浮点输入，因此，您需要额外执行预处理。

In [ ]:
# convert to float32
image_float32 = tf.cast(image, tf.float32) / 255.0

xyz_float32 = tfio.experimental.color.rgb_to_xyz(image_float32)

# convert back uint8
xyz = tf.cast(xyz_float32 * 255.0, tf.uint8)

print(xyz.shape, xyz.dtype)

plt.figure()
plt.imshow(xyz)
plt.axis('off')
plt.show()

### 将 RGB 转换为 YCbCr

最后，`YCbCr` 是在很多视频系统中使用的默认颜色空间。通过 `tfio.experimental.color.rgb_to_ycbcr` 即可转换为 `YCbCr`：

In [ ]:
ycbcr = tfio.experimental.color.rgb_to_ycbcr(image)

print(ycbcr.shape, ycbcr.dtype)

plt.figure()
plt.imshow(ycbcr, cmap='gray')
plt.axis('off')
plt.show()

不过，更有趣的一点是可以将 `YCbCr` 分解为 `Y'`（亮度）、`Cb`（蓝色差分色度）和 `Cr`（红色差分色度）组件，其中每个组件都携带有视觉意义的信息：

In [ ]:
y, cb, cr = ycbcr[:,:,0], ycbcr[:,:,1], ycbcr[:,:,2]

# Y' component
plt.figure()
plt.imshow(y, cmap='gray')
plt.axis('off')
plt.show()

# Cb component
plt.figure()
plt.imshow(cb, cmap='gray')
plt.axis('off')
plt.show()

# Cr component
plt.figure()
plt.imshow(cr, cmap='gray')
plt.axis('off')
plt.show()